### This Notebook will scrape MTG Goldfish's MetaGame Page for the top Modern Format decks
#### The purpose is to pull together a small data set to practice basic data analysis on

Importing Necessary Libraries and defining primary variables

In [2]:
#importing requests to access the websites that hold the meta data and beautiful soup that will help parce the html and csv to create a csv file for late data analysis
import requests
from bs4 import BeautifulSoup
import csv
#this holds the html text for the main page that holds the links for each meta deck 
main_page = requests.get('https://www.mtggoldfish.com/metagame/modern/full#paper')
#creatinga beautiful soup object of the main page text
main_soup = BeautifulSoup(main_page.content,'html.parser')
#list to hold the links to individual decks
links = []
#lists for holding the various data
decks = {}

Getting the pages for each specific deck from the metagame page

In [3]:
#finding each deck title (to grab the links)
titles = main_soup.find_all('div',class_ = 'archetype-tile-title')
#iterating through each title to save the links to each deck
for x in titles:
    title = x.find('span', class_ = 'deck-price-paper')
    link = 'https://www.mtggoldfish.com'+title.find('a').get('href')
    links.append(link)

Iterating through the list of deck pages to gather the desired data from each deck

In [4]:
#iterating through each deck link
for i in links:
    #holding the card names
    names = []
    #holding the card counts 
    counts = []
    #holding the card prices
    card_prices = []
    #requesting the text of the deck page
    deck_page = requests.get(i)
    #creating a soup object from the deck page
    deck_soup = BeautifulSoup(deck_page.content,'html.parser')
    #finding the total deck price and adding it to the list
    total_price = deck_soup.find('div',class_='deck-price-v2 paper').text
    total_price = float(str(total_price).replace('$','').replace(',',''))
    #finding the card names and adding them to the list
    soup_table = deck_soup.find('div',class_='deck-table-container')
    soup_names = soup_table.find_all('span',class_='card_id card_name')
    #iterating through the names to get the text and add it to the list
    for x in soup_names:
        name = x.find('a').get_text()
        names.append(name)
    #finding the card quantities+price and adding them to the list
    soup_counts = soup_table.find_all('td',class_='text-right')
    for x in soup_counts:
        count = x.get_text()
        if '$' in count:
            card_price = float(str(count).replace('$','').replace(',',''))
            card_prices.append(card_price)
        else:
            count = count.replace('tix','')
            counts.append(count.strip('\n'))
    #getting the deck name
    deck_name = deck_soup.find('h1',class_='title').get_text()
    deck_name = deck_name[1:deck_name.find('by')-1]
    #adding the data to decklists
    decks[deck_name] = {"total price":total_price,
                        "cards":names,
                        "counts":counts,
                        "card prices":card_prices
                       }

Taking the data and saving it to a csv for later analysis

In [6]:
#opening/making the file
f = open('Top Mordern Magic Decks Data.csv','w')
#initializing the writer
w = csv.writer(f)
#iterating through each deck
#Columns: Deck Name, Total Price, Card Name, Card Quantity, Card Price
counter = 0
w.writerow(['Deck Name', 'Total Price', 'Card Name', 'Card Quantity', 'Card Price'])
for deck in decks:
    for name in decks[deck]['cards']:
        w.writerow([deck,decks[deck]['total price'],name,decks[deck]['counts'][counter],decks[deck]['card prices'][counter]])
        counter += 1
    counter = 0